# Introduction

The goal of this project is to create a simulation that models granular forces between sand particles being dropped into a container with inwardly sloping walls. We'll create walls and drop a bunch of sand particles into the container to hopefully see something that resembles an hourglass.

My approach to starting this project was to start with something I already had working – the Lennard-Jones simulator that I've been working on throughout Gould Chapter 8. First, I removed the periodic boundary conditions and other unncessary methods. Then, I worked on getting particles to drop in in a grid lattice. Then, I added the walls, one at a time. The V shape was sufficient for the simulation, but I _really_ wanted an actual hourglass for the aesthetic – that took quite a bit of work, but after many wall iterations, the final hourglass drawing function is quite short.

Here are a few pictures of the evolution of the simulator and visualizations. The best 3 iterations of the simulator with "live action" visualization clients can be found here: https://github.com/tedtop/CSCI-477-Simulation/tree/master/Hourglass

See Methods below for more discussion on my hourglass simulator evolution.

<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_1.png" width="400"/>
<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_2.png" width="400"/>
<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_4.png" width="400"/>

![Complete "live action" simulator](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_5.png)

# Methods

### Simulator evolution

As I mentioned in the introduction, I took an iterative approach. Besides using git for tracking changes, I additionally saved separate simulators when I achieved a desired outcome. The "live action" visualizers are based on Jesse's animation driver where the animation's update function runs the simulation steps rather than animating pre-generated simulation data points. `hourglass_client.py` is the first fully functional "live action" visualization client for the `HourglassSimulator.py`. This one used Hooke's law as suggested in the assignment, and had early variations of wall drawing functions. I wasn't happy with the sand particles bouncing around like beach balls, so the second iteration `granular_simulator.py` uses `GranularHourglassSimulator.py` uses a Hertzian particle interaction model which greatly improved the visualization to look more realistic. For fun, I wanted a perpetual hourglass that respawns particles after they've reached the bottom which can be found in `respawning_example.py` and `RespawningHourglassSimulator.py`. The final iteration that's attached to make this notebook work is a combination of these 3; simplified and cleaned up as much as possible – and allows choosing Hooke's or Hertzian contact models – which was cool to be able to switch between the two while working on this writeup.

### Hooke's and Hertzian particle contact force calculations

![Hooke vs Hertzian Contact Forces Models](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/hooke_vs_hertzian.png)

Force Components:

* Normal forces push overlapping particles apart
* Damping forces reduce relative velocity to dissipate energy
* Friction forces resist sliding motion between particles
* Gravitational forces pull particles downward

The `compute_acceleration()` method is an entry point that selects the appropriate contact model based on simulator configuration and passes the necessary data to the specialized Numba-ized force calculation functions. Then, applies wall forces separately after particle-particle interactions. In this two-phase approach, forces are calculated first between particles, then with the walls by calling `apply_wall_constraints()`.

Each force calculation function (`compute_particle_forces_hertzian()` and `compute_particle_forces_hooke()`) works similarly (they are set up like this by design to be Numba-ize-able).

(0) Reset forces and apply gravity to all particles

(1) We iterate through all particle pairs, ensuring each particle pair is processed exactly once (i,j but not j,i)

(2) calculate separation and check for overlap; compute the vector from i to j, compute the distance between particle centers, determine the sum of their radii, check if they overlap by comparing distance to sum of radii, only proceed with force calculation if overlap exists

(3) calculate normalized direction vector and overlap amount

(4) calculate relative velocity components; decompose the relative velocity into normal and tangential components; compute the relative velocity vector between particles, project this velocity onto the normal direction (dot product), subtract the normal component to get the tangential component, calculate the magnitude of the tangential velocity.

(5a) Calculate the normal force (Hertzian):

**Hertzian contact model (non-linear spring):**
$$F_{\text{normal}} = k\sqrt{\delta} - \gamma v_n$$

Where:
- $k$ is the spring constant
- $\delta$ is the overlap distance between particles
- $\gamma$ is the damping coefficient
- $v_n$ is the normal component of relative velocity

Applies the Hertzian contact model where the force is proportional to the square root of overlap, calculates a damping force proportional to normal velocity, only applies damping when particles are approaching each other, combines spring and damping forces to get the total normal force.

(5b) Calculate the normal force (Hooke's):

**Hooke's law model (linear spring):**
$$F_{\text{normal}} = k\delta - \gamma v_n$$

Where the variables have the same meaning, but note the linear relationship with $\delta$ rather than the square root relationship.

Applies Hooke's law where force is directly proportional to overlap, calculates damping based on the dot product of the velocity and direction, projects both forces along the normal direction, combines the spring and damping forces to get the total force components.

Note: The Hertzian implementation calculates a scalar force magnitude first and then applies direction, while the Hooke's implementation computes force vector components directly, but both approaches produce physically equivalent results with different computation sequences.

(6) Calculate tangential (friction) force: apply tangential friction forces between particles using `f_t = min(friction_coef * abs(f_n), gamma * v_t_mag)`, which implements Coulomb's friction law with an upper limit proportional to normal force; this friction mechanism is useful for creating realistic granular behavior as it allows particles to grip each other and form stable structures rather than sliding freely like perfect spheres would in a frictionless environment

(7) Combine forces and update accelerations

(8) update potential energy; note that the potential energy formulas also differ:

**Hertzian potential energy:**
$$U_{\text{Hertzian}} = \frac{2}{5}k\delta^{5/2}$$

**Hooke's law potential energy:**
$$U_{\text{Hooke}} = \frac{1}{2}k\delta^2$$

(9) Finally, after all particle-particle interactions are processed, wall forces are applied (described in wall collision detection section below)

The key differences between Hertzian and Hooke's: (1) the critical difference is that Hertzian forces scale with the square root of overlap, while Hooke's law forces scale linearly, (2) the Hertzian implementation separates normal and tangential forces, while the Hooke's law implementation separates spring and damping forces, (3) the Hertzian model only applies damping when particles approach (v_n < 0), while the Hooke's law model applies it more generally, (4) different potential energy formulas.

### Accelerating force calculations with Numba

Force calculations between particles are computationally intensive, particularly as the number of particles increases. I wanted to improve my Lennard-Jones simulator with Numba before the exam. I had already decoupled my simulator and visualizer to run outside Jupyter notebooks, which was a risky move the night before an exam. I didn't want introduce something else I was unfamiliar with.

With this project, once I started increasing the number of particles, the simulation started getting very laggy and I decided it was time to implement Numba's just-in-time (JIT) compilation to dramatically accelerate these calculations. The @njit decorator is applied to the force calculation methods, which compiles these functions to optimized machine code at runtime.
The most significant performance benefits come from JIT-compiling the nested loops that calculate pairwise interactions between particles. In the "live action" client, before Numba, N=100 was fine, N=200 was ok, and at N=300 the simulation slowed to a crawl. With Numba, I could get up to N=2000 particles before noticing the same crawl. A 10x improvement by visual inspection. However, the placement of thousands of particles with the rejection sampling placement algorithm now caused a delay in launching the visualizer.

2000 Particles with Numba:

![2000 Particles with Numba](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/numba_2000b.png)

### Drawing the hourglass walls

My first attempt at the walls came out goofy, like an invisible container. From there I iterated to a left ramp, then a right ramp, then two ramps. I wasn't happy with the shape. I decided to bring the walls up to form a V in the top half and reflected it to form an upside-down V in the bottom half. I made this design choice because I wanted to keep the hourglass shape constant, but with the ability to vary the neck width. In the final iteration, the mess of wall drawing functions are simplified to one function that creates 4 wall segments defined by their top and bottom coordinates, with the neck positioned at half the container height.

<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/invisible_container.png" width="400"/>
<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_2.png" width="400"/>
<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/intro_3.png" width="400"/>


### Wall collision detection

![Wall Collision Forces](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/wall_collision_forces.png)

`apply_wall_constraints()` handles the bottom wall as a special case. The bottom wall is horizontal and requires simpler calculations than the sloped walls. It either respawns particles or applies forces to bounce them off the bottom. If respawn_particles is enabled, particles that hit the bottom are repositioned at the top It then delegates to `handle_wall_segments()` for more complex sloped walls that form the hourglass shape.

For each particle, `handle_wall_segments()` (a) determine if the particle is within a wall segment's y-range, (b) calculate the wall position at the particle's current height; a linear interpolation that finds the wall's x-position at the particle's y-coordinate, (c) calculate the wall normal (pointing away from the wall) and tangent (pointing along the wall) vectors; these are important for separating particle velocities in to normal and tangential components which receive different treatments, (d) checks for wall penetration as a safety measure; particles that penetrate too deeply are repositioned to prevent them from passing through the walls, (e) decompose velocity and apply appropriate forces – spring force based on penetration depth, damping force in the normal direction, and friction in the tangential direction based on Coulomb's law.

The wall collision system models several important physics concepts: (a) elastic deformation - spring forces model the compression of particles against the walls, (b) energy dissipation - damping and restitution reduce energy with each collision, (c) surface friction - Coulomb friction prevents unrestricted sliding along walls.

### Spring and damping constants

The spring constant k and damping coefficient gamma are critical parameters that determine the physical behavior of the granular material. The spring constant (k) controls the stiffness of particle interactions, with higher values creating more rigid, less compressible particles. Lower values created more bouncy particles that were too unrealistic for an hourglass. I worked with k=20.0 for the most part. While there was more particle overlap than I would have liked, values like k=50.0 and k=100.0 would more often than not crash the simulation. The damping coefficient (gamma) influences energy dissipation during collisions, with higher values reducing particle bouncing. I played around with the k and gamma values quite a bit (lack of lab notes / documenation discussed in Interpretation) in the first iteration `hourglass.py` where I used Hooke's law, but every combination I tried had particles bouncing around like beach balls in the top and bottom chambers, or flowing down the walls and and straight out like a liquid without forming any sign of arching structures whatsoever – which was what led me to find the Hertzian contact model.

### Friction and restitution coefficients

In the process of trying to reduce the bouncing beach balls behavior, I tried adding a restitution coefficient to the walls to reduce the amount of energy conserved during the wall bounce interactions. I started with a hard coded 0.9 and then had to refactor that variable out into the simulator constructor parameters and eventually settled on 0.3 used throughout the rest of my experimentation – meaning only 30% of kinetic energy is preserved in the normal direction during a collision.

Similarly, to attempt to mimic more realistic sand, I introduced the friction coefficient to add sliding friction with the walls. While, damping (gamma) acts primarily along the normal direction (compression/expansion), the friction coefficient acts in the tangential (perpendicular to normal) direction. Damping represents energy dissipation during compression/decompression and friction represents resistance to sliding.

I didn't experiment too much with these other than recognizing that introducing them scaled down the forces (friction_coef=0.5, restitution_coef=0.3) sufficiently for me to see the improvement I was looking for in the animation.

### Particle placement

The particle placement system initializes particles in physically valid positions at the top of the hourglass. The `initialize_random_falling_particles()` method places particles within the upper portion of the container while ensuring no initial overlaps occur between particles by maintaining a minimum distance of 2.2 times the particle radius. The placement algorithm uses a rejection sampling approach, attempting new random positions until a valid non-overlapping position is found or a maximum attempt count is reached. This method also supports particle respawning, reintroducing particles that exit the bottom of the hourglass back to the top, enabling continuous flow simulations.

![Hooke vs Hertzian Contact Forces Models](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/respawn_zone.png)


### Verlet integration method

My starting point for this project was the most polished, latest iteration of my Lennard-Jones simulator. Likewise, this simulator uses the Velocity Verlet integration algorithm, which has suited me well for the molecular dynamics simulations in Gould Chapter 8. This second-order method updates positions and velocities in a two-stage process: first advancing velocities by a half-step, then updating positions using these half-step velocities, calculating new accelerations based on the updated positions, and finally completing the velocity update with another half-step. The Velocity Verlet algorithm offers excellent energy conservation properties compared to simpler methods like Euler integration, reducing numerical drift in long simulations.

### Complete simulator

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.animation import FuncAnimation

# Increase animation embed limit
mpl.rcParams['animation.embed_limit'] = 500  # In MB

# Import the simulator and visualizer classes
from HourglassSimulator import HourglassSimulator
from HourglassVisualizer import HourglassVisualizer

# Create the simulator with appropriate parameters
simulator = HourglassSimulator(
    N=250,                   # Number of particles
    Lx=20.0,                 # Box width
    Ly=20.0,                 # Box height
    temperature=10.0,        # Initial temperature 10.0 to answer parts a and b
    dt=0.005,                # Time step size
    gravity=5.0,             # Gravity strength
    particle_radius=0.4,     # Particle radius
    k=50.0,                  # Spring constant
    gamma=5.0,               # Damping coefficient
    contact_model="hertzian", # Use Hertzian contact model (alternative: "hooke")
    neck_width=2.5,          # Width of hourglass neck
    wall_width=0.5,          # Wall thickness
    friction_coef=0.5,       # Friction coefficient
    restitution_coef=0.3,    # Coefficient of restitution
    respawn_particles=False  # Don't respawn particles
)

# Create the hourglass shape
simulator.draw_hourglass()

# Initialize particles at the top
simulator.initialize_random_falling_particles()

# Create a visualizer
visualizer = HourglassVisualizer(simulator)

# Create animation for the first x seconds
animation, fig = visualizer.create_animation(
    limited_duration=20.0,  # Run for x simulation seconds
    steps_per_frame=10,     # Number of simulation steps per frame
    interval=30             # Delay between frames in milliseconds
)

# Display animation
animation_html = animation.to_jshtml()
plt.close(fig)
HTML(animation_html)

# Analysis

### How long does it take for the kinetic temperature to decrease to 10% of its initial value?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from HourglassSimulator import HourglassSimulator
from HourglassVisualizer import HourglassVisualizer

# Create the simulator with the same parameters
simulator = HourglassSimulator(
    N=200,
    Lx=20.0,
    Ly=20.0,
    temperature=10.0,
    dt=0.005,
    gravity=5.0,
    particle_radius=0.5,
    k=20.0,
    gamma=2.0,
    contact_model="hertzian",  # change this between "hertzian" and "hooke"
    # contact_model="hooke",  # change this between "hertzian" and "hooke"
    neck_width=2.5,
    wall_width=0.5,
    friction_coef=0.5,
    restitution_coef=0.3,
    respawn_particles=False
)

# Create the hourglass shape
simulator.draw_hourglass()

# Initialize particles at the top
simulator.initialize_random_falling_particles()

# Set velocities to match initial temperature
simulator.set_velocities()

# Verify initial temperature
simulator.compute_metrics()
initial_temp = simulator.temperature
target_temp = initial_temp * 0.1
print(f"Initial temperature: {initial_temp:.4f}")
print(f"Target temperature (10%): {target_temp:.4f}")

# Keep track of time and temperatures
times = []
temps = []

# Run simulation until temperature drops to target or max time reached
max_time = 20.0  # Maximum simulation time
time_step = 0.05  # Record data every 0.05 time units
steps_per_record = int(time_step / simulator.dt)

found_target = False
while simulator.time < max_time:
    # Run simulation for time_step
    for _ in range(steps_per_record):
        simulator.step()

    times.append(simulator.time)
    temps.append(simulator.temperature)

    # Check if temperature has dropped to target
    if simulator.temperature <= target_temp and not found_target:
        found_target = True
        target_time = simulator.time
        print(f"Temperature dropped to {simulator.temperature:.4f} at time {target_time:.2f}")

        # Create a snapshot to visualize the particle distribution
        simulator.take_snapshot()

# Plot temperature vs time
plt.figure(figsize=(10, 6))
plt.plot(times, temps)
plt.axhline(y=target_temp, color='r', linestyle='--', label='10% of initial temperature')
if found_target:
    plt.axvline(x=target_time, color='g', linestyle='--', label=f'Target reached at t={target_time:.2f}')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Temperature vs Time')
plt.legend()
plt.grid(True)
plt.show()

# If we found the target temperature, visualize the particle distribution
if found_target:
    # Create visualizer and plot the distribution
    visualizer = HourglassVisualizer(simulator)
    fig = visualizer.plot_snapshots(times=[target_time], rows=1, cols=1, figsize=(8, 8))
    plt.show()

    # Calculate basic distribution statistics
    last_snapshot = simulator.snapshots[-1]
    positions = last_snapshot['positions']

    # Count particles in different regions
    top_half_count = np.sum(positions[:, 1] > simulator.Ly/2)
    bottom_half_count = np.sum(positions[:, 1] <= simulator.Ly/2)

    # Define neck region
    neck_y = simulator.Ly / 2
    neck_width = simulator.neck_width
    neck_x_min = (simulator.Lx - neck_width) / 2
    neck_x_max = neck_x_min + neck_width

    # Count particles in the neck region (±1 unit around neck height)
    neck_region_count = np.sum((positions[:, 1] > neck_y - 1) &
                              (positions[:, 1] < neck_y + 1) &
                              (positions[:, 0] > neck_x_min) &
                              (positions[:, 0] < neck_x_max))

    # Count particles at rest at the bottom
    near_bottom_count = np.sum(positions[:, 1] < 2.0)

    print(f"\nParticle distribution at time {target_time:.2f}:")
    print(f"Total particles: {simulator.N}")
    print(f"Particles in top half: {top_half_count} ({top_half_count/simulator.N:.1%})")
    print(f"Particles in bottom half: {bottom_half_count} ({bottom_half_count/simulator.N:.1%})")
    print(f"Particles near neck region: {neck_region_count} ({neck_region_count/simulator.N:.1%})")
    print(f"Particles settled at bottom: {near_bottom_count} ({near_bottom_count/simulator.N:.1%})")

<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/part_a_hooke.png" width="600"/>
<img src="https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/part_a_hertzian.png" width="600"/>

The images above show the difference in time it took to reach 10% of initial temperature. Hooke's on the left. Hertzian on the right.

All other parameters held constant, using the Hooke's model for contact forces, repeated simulations show the time to reach from initial temperature 10 to 1 takes about 8 seconds. With the Hertzian model the time to reach from temperature 10 → 1 takes around 13-15 seconds. By the time every simulation had reached the target temperature, all particles have passed through the neck of the bottle and settled at the bottom.

In my early iterations of the simulator using Hooke's law I observed the particles bouncing around like beach balls no matter how much I tweaked the spring and damping constants $k$ and $\gamma$ (see Interpretation for improvement). Through experimentation with other force models I found the Herzian contact force model produced the most visually appealing animations. The Hertzian contact model creates more realistic and slower sand flow than Hooke's law because its non-linear force response – proportional to δ^(1/2) rather than δ – better captures the physics of granular materials. I was able to visually observe sand particles building up more stable formations in the top chamber of the hourglass rather than bouncing around or flowing out like water like they did under Hooke's law. This is also evident in the above simulation by switching between "hertzian" and "hooke". With "hooke" ~70% of particles settled in the bottom detection region versus ~50% with "hertzian" – indicating more overlap of particles with Hooke's.

## Compute the mean kinetic temperature versus time averaged over three runs. What functional form describes your results for the mean kinetic temperature at long times?

![Kinetic Temperature vs Time](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/kt_versus_time.png)

![Kinetic Temperature Power Law Analysis](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/kt_power_law_analysis.png)

The mean kinetic temperature in our granular hourglass simulation follows a power law decay at long times, approximated by $T(t) \approx A \cdot t^{-\alpha}$, rather than an exponential decay. Physically, $A$ relates to the initial energy state of the system before entering the pure power-law regime and $\alpha$ is the power law exponent that characterizes the system's energy dissipation rate. This behavior is characteristic of granular materials where energy dissipation occurs through multiple mechanisms: inelastic collisions between particles, friction with walls, damping effects, and geometric constraints from the hourglass shape. The simulation shows an initial temperature increase as particles begin falling and convert potential energy to kinetic energy, followed by a rapid decrease as they interact at the neck, and finally a slower power law decay as they settle at the bottom of the hourglass. 

### With a fluid, the rate the fluid leaves a funnel is proportional to the height of the fluid. In an hour glass, the rate the sand grains fall is constant. Why?

I vaguely recall an experiment from AP Physics that we wrote an extensive lab about water flowing out of a funnel. I attempted to write a computer simulation to model what we were actually observing, but nothing I tried worked. In my rudimentary computer simulation the water flowed out at a constant rate though our lab measurements and derived equations showed that the rate of flow was proportional to the height of the water. At the time I didn't have the knowledge of how to do these time step based simulations where forces (or in this case the flow rate based on the height of the water) were recalculated every time step. Unfortunately, it's in an attic in Florida or I would have dug it out to compare results with. It looks like what I would have wanted to model with my water simulation is Torricelli's law.

In granular materials like sand, particles form arch-like structures that support the weight of particles above them. This means that pressure at the bottom of a sand column doesn't increase with height, as it would in a fluid. The walls of the hourglass help support the grains through friction, creating friction chains,which redistributes the forces like a stone arch bridge. The flow rate through the neck of the hourglass is primarily determined by the width of the aperture rather than pressure. Sand grains interact through contact forces and friction. When particles try to pass through the narrow neck, they form temporary arches and experience complex interactions that regulate the flow rate.

Unlike fluids where flow depends on the height of the fluid column, sand in an hourglass falls at a nearly constant rate – at least compared to water flow rate that increases with the square root of the water height. Counterintuitively to the expection of seeing constant flow, the flow rate lines in the graphs below reflect a linearly decreasing flow rate. If we were to crop the flow rate lines to just the active portions of each of the simulations we could fit a linear regression line. I suspect this is because I generated these graphs using N=200 particle simulations and we quickly run out of particles to keep the top chamber full of interactions. With fewer particles remaining the flow rate decreases nearly linearly. This accurately simulates the depletion phase of a real hourglass (see Interpretation for improvement).

![Flow vs Opening Width](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/flow_vs_opening_width.png)

Neck Width – Wider neck openings allow more particles to pass through simultaneously, increasing the overall flow rate. But for each fixed width, the flow pattern remains relatively constant.

![Flow vs Particle Size](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/flow_vs_particle_size.png)

Particle Size: Smaller particles flow more quickly through the same opening size. When particles become too large relative to the opening, intermittent jamming creates irregular flow patterns.

![Flow vs Damping](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/flow_vs_damping.png)

Damping: Higher damping reduces flow by absorbing kinetic energy during collisions and creating stronger inter-particle force networks.

![Flow vs Spring Constant](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/flow_vs_spring_constant.png)

Spring Constant: Stiffer particles (higher k) show more jamming behavior and slower overall flow. This is because stiffer particles maintain their shape more rigidly when forming arches.

# Interpretation

My conclusion from working on this project, writing the analysis, and this course as a whole is that it's really hard to simulate physical systems. I'll share my takeaways from working on the writeup:

* I should have kept notes of early experimentation with my "live action" simulators. It only occurred to me while working on the write up how much I wish I had kept a lab notebook while working on my simulators. Specifically, what really held me up was not being able to refer back to what kinds of observations varying the values of k and gamma produced when I introduced them into the simulator. In my early simulations using Hooke's law, I found combinations of k and gamma that produced wildly bouncy particles that flew around like beach balls in both chambers, and other values that produced animations that flowed through the neck like water; showing almost no repelling forces with no opportunity to build any sort of self-supporting structure.

* In the final iteration of the simulator (that renders embedded videos to the notebook), certain combinations of parameters would trigger a seemingly unrelated error somewhere deep in the matplotlib animation function. This was almost certainly some numerical instability – (my dt was 0.005) for all simulations (after I upgraded to Numba). Some embedded animations would just freeze up prematurely – almost certainly numerical instability. Whereas with the N-Body simulations we'd see the particles shoot off into infinity, the causes for these numerical instabilities were not as apparent, nor could I think of a way to debug them other than to dial down whichever parameter caused the freeze or this RGBA error.

![RGBA error](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/rgba_error.png)

* Certainly related to the numerical instability mentioned above, while working in this notebook, many of my embedded simulations would freeze up and the animation would continue in a frozen state. This never happened with my "live action" simulator clients. They would all complete and the particles would all come to a formation in the bottom chamber in around 12 seconds. With the embedded animations, I would set the animation to run for some number of seconds, but I would end up with some "dead air" at the end - meaning the simulation had reached the desired time in seconds, but the animation frames would continue. Not a huge deal, but it did make me question if there was some bug related to the different time related variables (dt, time, step_count, snapshot_interval, frames, interval, limited_duration, steps_per_frame, max_time, time_window) causing a discrepancy with the time scales being synchronized. I was not able to track this down.

* I noticed a bug/discrepancy in the way the width of the neck would appear in the visualizations with neck_width=1.0 didn't seem like the particles should be falling through, but I did investigate this to my satisfaction that the neck was in fact wide enough for the particles to pass through. At neck_width=1.0 the polygons are drawn in a way that looks like there isn't an opening, but it's there. I realized this has to do with with wall width and the necessary wall penetration correction. I could add a check for numerical instability and an adaptive timestep when large accelerations are detected. I decided to leave it as is for now.

![RGBA error](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/tiny_neck_size.png)

Further investigation:

* The deeper I got into this the more avenues of further investigation presented themselves. I came across the concept of force chains and arches that would have been interesting to investigate further numerically. The Janssen Effect: This is the formal name for the pressure saturation phenomenon in granular materials. After a certain depth, additional sand height doesn't increase pressure at the bottom due to wall friction supporting the weight. The possibility of quantitative analysis and comparison of liquids under Torricelli's law and granular materials and the parameters that affect those flows.

* It would be interesting to investigate further into flow rates. I wasn't sure of my analysis of my flow rate graphs. I thought they looked cool, but while answering the question, remembered that a constant flow should have been a horizontal line, not a linear decay. After some research, I came to the realization that this simulation models the decay phase of how a real hourglass would empty. This made me ponder how to scale this simulation to many thousand particles to see if I would indeed be able to reproduce a constant flow rate like in a real hourglass – the whole reason why they are so good at keeping time.

* Lastly, throughout the process of building the simulator, I kept extracting more and more variables into the constructor which started looking pretty impressive. With so many parameters now more easily configurable, I found myself getting confused about what effects varying different combinations would have. Some later simulations with higher k values (based on a suggestion we talked about in class) that produced nice arching structures in the top chamber would have some particles falling down naturally as expected and others would slide down the top walls and unnaturally slide against the opposite wall of the bottom chamber. Neither turning up the friction nor decreasing the neck size seemed to help. It made me consider how to take a more systematic approach to testing and optimizing so many combinations of parameters to achieve a more realistic hourglass. Perhaps, the next step is to apply some machine learning to minimizing the flow rate. I tried creating a few thousand really small particles. What I really wanted to see was a pyramidal mound forming in the bottom chamber.

![Particle energy coloring](https://github.com/tedtop/CSCI-477-Simulation/raw/master/Hourglass%20Project/images/particle_energy_coloring.png)